## Setup Spark with Iceberg and MinIO
This configures a Spark session with Iceberg support and sets up the lake catalog to use MinIO as the warehouse (s3a://my-bucket/iceberg-lakehouse).
We're using HadoopCatalog and enabling S3A access with local MinIO credentials.

In [1]:
from pyspark.sql import SparkSession

packages = ",".join([
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.2", # Iceberg runtime for Spark 3.5
    "org.apache.hadoop:hadoop-aws:3.3.4",  # Hadoop support for AWS S3 (S3A)
    "com.amazonaws:aws-java-sdk-bundle:1.12.530",  # AWS SDK bundle (used under the hood)
])

spark = (
    SparkSession.builder
      .appName("Iceberg Lakehouse on MinIO")
      .config("spark.jars.packages", packages)
      .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") # Enable Iceberg SQL support
      .config("spark.sql.defaultCatalog", "lake") # Use 'lake' as the default catalog for Spark SQL

      # Define the 'lake' Iceberg catalog using HadoopCatalog backed by MinIO 
      .config("spark.sql.catalog.lake", "org.apache.iceberg.spark.SparkCatalog")
      .config("spark.sql.catalog.lake.type", "hadoop")
      .config("spark.sql.catalog.lake.warehouse", "s3a://my-bucket/iceberg-lakehouse")

      # MinIO (S3-compatible) connection configs
      .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
      .config("spark.hadoop.fs.s3a.path.style.access", "true") # Enable path-style access (important for MinIO)
      .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") # Disable SSL for local setup
      .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
      .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
      .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

      .getOrCreate()
)

print("Spark:", spark.version)


Spark: 3.5.3


## Create Iceberg Table from Existing Parquet Files

We switch to the lake catalog, create the nyc namespace and define a new Iceberg table nyc.taxis by loading two raw Parquet files from MinIO.
This creates a managed Iceberg table that supports versioning, time travel and schema evolution from the start.

In [2]:
spark.catalog.setCurrentCatalog("lake")
spark.sql("CREATE NAMESPACE IF NOT EXISTS nyc")

spark.sql("""
CREATE TABLE IF NOT EXISTS nyc.taxis
USING iceberg
AS
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-05.parquet`
UNION ALL
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet`
""")


DataFrame[]

## Preview a few rows from nyc.taxis
Run a quick Spark SQL sanity check to verify the table is readable and the expected columns are present.

In [3]:
spark.sql("""
SELECT tpep_pickup_datetime, tpep_dropoff_datetime, VendorID, passenger_count, total_amount  FROM nyc.taxis limit 5;
""").show(truncate=True)

+--------------------+---------------------+--------+---------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|VendorID|passenger_count|total_amount|
+--------------------+---------------------+--------+---------------+------------+
| 2025-05-01 00:07:06|  2025-05-01 00:24:15|       1|              1|        29.0|
| 2025-05-01 00:07:44|  2025-05-01 00:14:27|       2|              1|       18.65|
| 2025-05-01 00:15:56|  2025-05-01 00:23:53|       2|              1|       15.75|
| 2025-05-01 00:00:09|  2025-05-01 00:25:29|       2|              1|       71.94|
| 2025-05-01 00:45:07|  2025-05-01 00:52:45|       2|              1|       17.25|
+--------------------+---------------------+--------+---------------+------------+



## View Table History for nyc.taxis
Iceberg keeps track of table version history, including metadata about schema changes, snapshot lineage, and when each version became active. In snapshot_id you can see your first snapshot id and file for it you can find s3://my-bucket/iceberg-lakehouse/nyc/taxis/metadata/snap-{snapshot_id}-**.avro

In [4]:
spark.sql("""
SELECT * FROM nyc.taxis.history ORDER BY made_current_at DESC;
""").show(truncate=False)


+-----------------------+-------------------+---------+-------------------+
|made_current_at        |snapshot_id        |parent_id|is_current_ancestor|
+-----------------------+-------------------+---------+-------------------+
|2025-08-11 16:00:43.386|8062238580457074585|NULL     |true               |
+-----------------------+-------------------+---------+-------------------+



## Check Data Files in nyc.taxis
This shows the list of files in the Iceberg table, including:

file_path: where each file is stored
record_count: how many rows in each file
file_size_in_bytes: file size
content: type of data (e.g. data, delete)


In [5]:
spark.sql("""
SELECT content, file_path, record_count, file_size_in_bytes
FROM nyc.taxis.files;
""").show(truncate=False)


+-------+--------------------------------------------------------------------------------------------------------------+------------+------------------+
|content|file_path                                                                                                     |record_count|file_size_in_bytes|
+-------+--------------------------------------------------------------------------------------------------------------+------------+------------------+
|0      |s3a://my-bucket/iceberg-lakehouse/nyc/taxis/data/00001-3-22c9cd14-c318-4214-b7ef-3a349bd60b7f-0-00001.parquet |1048576     |16886663          |
|0      |s3a://my-bucket/iceberg-lakehouse/nyc/taxis/data/00003-5-22c9cd14-c318-4214-b7ef-3a349bd60b7f-0-00001.parquet |1048576     |16960598          |
|0      |s3a://my-bucket/iceberg-lakehouse/nyc/taxis/data/00006-8-22c9cd14-c318-4214-b7ef-3a349bd60b7f-0-00001.parquet |1048576     |16872467          |
|0      |s3a://my-bucket/iceberg-lakehouse/nyc/taxis/data/00009-11-22c9cd14-c318-4

## Simulate a New Batch Ingestion
This cell simulates a new batch arriving by inserting a random 10% sample of rows from the June dataset into the nyc.taxis table.
After the insert, the history table is queried again to show that a new snapshot has been added.

This demonstrates how Iceberg creates a new table version with each write, enabling time travel and version tracking.

In [6]:
spark.sql("""
INSERT INTO nyc.taxis
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet` WHERE rand() < 0.10;
""")

spark.sql("""
SELECT * FROM nyc.taxis.history ORDER BY made_current_at DESC;
""").show(truncate=False)


+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-08-11 16:00:45.126|4274936221537877971|8062238580457074585|true               |
|2025-08-11 16:00:43.386|8062238580457074585|NULL               |true               |
+-----------------------+-------------------+-------------------+-------------------+



## Save Snapshot IDs for Time Travel
Two snapshot IDs are copied from the nyc.taxis.history table to use in the next steps:

In [7]:
snapshot_id_1='8062238580457074585'
snapshot_id_2='4274936221537877971'

## Compare Row Counts Between Snapshots
This runs two queries to count the number of rows in the nyc.taxis table at two different points in time — before and after the simulated batch ingest.

The lower snapshot ID represents the original state of the table.
The higher one reflects the updated state after inserting 10% more data.

In [8]:

spark.sql(f"""
SELECT count(*) FROM nyc.taxis VERSION AS OF {snapshot_id_1} LIMIT 5;
""").show(truncate=False)

spark.sql(f"""
SELECT count(*) FROM nyc.taxis VERSION AS OF {snapshot_id_2} LIMIT 5;
""").show(truncate=False)


+--------+
|count(1)|
+--------+
|8914805 |
+--------+

+--------+
|count(1)|
+--------+
|9347281 |
+--------+



## Roll Back Table to an Earlier Snapshot

This command reverts the nyc.taxis table back to a previous state using the older snapshot ID.
It’s a simple way to undo changes and return the table to how it looked before the last ingestion.

In [9]:
spark.sql(f""" 
CALL lake.system.rollback_to_snapshot('nyc.taxis', {snapshot_id_1})
""")


DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]

## Verify the Rollback
This query checks the nyc.taxis.history table again to confirm that the rollback took effect.
A new snapshot should appear at the top, pointing back to the older snapshot ID.

In [10]:

spark.sql("""
SELECT * FROM nyc.taxis.history ORDER BY made_current_at DESC;
""").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-08-11 16:01:07.283|8062238580457074585|NULL               |true               |
|2025-08-11 16:00:45.126|4274936221537877971|8062238580457074585|false              |
|2025-08-11 16:00:43.386|8062238580457074585|NULL               |true               |
+-----------------------+-------------------+-------------------+-------------------+



## Check Row Count After Rollback
This query returns the current number of rows in the nyc.taxis table, after the rollback.
It should match the count from the older snapshot (before the extra batch was inserted).

In [11]:
spark.sql("""
SELECT count(*) FROM nyc.taxis  ;
""").show(truncate=False)

+--------+
|count(1)|
+--------+
|8914805 |
+--------+



## Add and Populate a New Column
This sequence adds a new column fare_bucket to the nyc.taxis table, classifying rides based on the total_amount.
The column is added after total_amount, then populated with a basic categorization logic:

'low' for fares under 10

'mid' for fares between 10 and 30

'high' for fares above 30

Finally, the results are grouped by fare_bucket to show the distribution:

In [12]:


spark.sql("""ALTER TABLE nyc.taxis ADD COLUMN fare_bucket STRING AFTER total_amount;
""")

spark.sql("""-- update a derived field to prove reads see the new column
UPDATE nyc.taxis
SET fare_bucket = CASE
  WHEN total_amount < 10 THEN 'low'
  WHEN total_amount < 30 THEN 'mid'
  ELSE 'high'
END
WHERE total_amount IS NOT NULL;
""")

spark.sql("""SELECT fare_bucket, COUNT(*) FROM nyc.taxis GROUP BY fare_bucket;
""").show(truncate=False)


+-----------+--------+
|fare_bucket|count(1)|
+-----------+--------+
|low        |765654  |
|mid        |5743528 |
|high       |2405623 |
+-----------+--------+



## Evolve Iceberg Schema On Write with `accept-any-schema`

In Iceberg, new columns can be added not only through `ALTER TABLE`, but also automatically during writes.

To enable this behavior, we switch the table to flexible write mode:

- Drop the `fare_bucket` column to simplify the schema before adding something new.
- Set the table property `'write.spark.accept-any-schema' = true` to allow appending data with previously unseen columns.
- Add a new derived column `tip_rate_pct` directly during ingestion — no schema pre-declaration needed.
- Use `.option("mergeSchema", "true")` to tell Iceberg to evolve the schema automatically.


In [13]:
import uuid
from pyspark.sql.functions import col, when, lit, udf, round as sround
from pyspark.sql.types import StringType


spark.sql("ALTER TABLE lake.nyc.taxis DROP COLUMN fare_bucket")
spark.sql("""ALTER TABLE lake.nyc.taxis SET TBLPROPERTIES (
  'write.spark.accept-any-schema'='true'
)
""")

# Load the raw parquet file
df = spark.read.parquet("s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet")

# Compute tip_rate_pct as a percentage of the total fare
df_with_tip = (
    df.withColumn(
        "tip_rate_pct",
        when(col("total_amount").isNull(), lit(None).cast("double"))
        .when(col("total_amount") <= 0, lit(None).cast("double"))
        .otherwise(sround(col("tip_amount") / col("total_amount") * 100.0, 2))
    )
)

(df_with_tip.writeTo("lake.nyc.taxis")
  .option("mergeSchema", "true")  # Enables schema evolution on write
  .append())

spark.sql("DESCRIBE TABLE lake.nyc.taxis;").show(30)

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|            VendorID|          int|   NULL|
|tpep_pickup_datetime|timestamp_ntz|   NULL|
|tpep_dropoff_date...|timestamp_ntz|   NULL|
|     passenger_count|       bigint|   NULL|
|       trip_distance|       double|   NULL|
|          RatecodeID|       bigint|   NULL|
|  store_and_fwd_flag|       string|   NULL|
|        PULocationID|          int|   NULL|
|        DOLocationID|          int|   NULL|
|        payment_type|       bigint|   NULL|
|         fare_amount|       double|   NULL|
|               extra|       double|   NULL|
|             mta_tax|       double|   NULL|
|          tip_amount|       double|   NULL|
|        tolls_amount|       double|   NULL|
|improvement_surch...|       double|   NULL|
|        total_amount|       double|   NULL|
|congestion_surcharge|       double|   NULL|
|         Airport_fee|       double|   NULL|
|  cbd_con

## Enable Day-Based Partitioning (Start of Partition Evolution Test)
This step adds a day-based partitioning rule on tpep_pickup_datetime.
It's the first step in testing partition evolution — a unique feature of Apache Iceberg that allows changing partition strategies over time without rewriting existing data.

In [14]:
spark.sql("ALTER TABLE nyc.taxis ADD PARTITION FIELD days(tpep_pickup_datetime)")


DataFrame[]

## Check Existing Partitions
This shows how the nyc.taxis table is currently partitioned.

Right now, the result shows partition = {NULL} and spec_id = 0, which means all the data was written before partitioning was added. There are no partitions yet — everything is stored together.

This confirms the starting point before testing partition evolution.

In [15]:
spark.sql("SELECT partition, record_count, file_count, last_updated_snapshot_id  FROM nyc.taxis.partitions").show(truncate=False)


+---------+------------+----------+------------------------+
|partition|record_count|file_count|last_updated_snapshot_id|
+---------+------------+----------+------------------------+
|{NULL}   |13237765    |15        |4331219133962302771     |
+---------+------------+----------+------------------------+



## Set Write Strategy for New Data
These commands configure how new data will be written into the nyc.taxis table:

WRITE DISTRIBUTED BY PARTITION tells Iceberg to group data by partition when writing files (useful after enabling partitioning).

WRITE ORDERED BY tpep_pickup_datetime, VendorID sorts rows within each partition by pickup time and vendor ID.

This helps optimize file layout for better query performance.

In [16]:
spark.sql("ALTER TABLE nyc.taxis WRITE DISTRIBUTED BY PARTITION")
spark.sql("ALTER TABLE nyc.taxis WRITE ORDERED BY tpep_pickup_datetime, VendorID")

DataFrame[]

## Rewrite Existing Data Files with New Partitioning
This command rewrites the existing data in the nyc.taxis table using Iceberg’s rewrite_data_files procedure.

It applies the new day-based partitioning and improves file layout by:

Compacting small files (min-input-files = 1)

Targeting ~128MB file sizes

Allowing partial progress commits in case of failures

This step brings older unpartitioned data in line with the new table configuration.

In [17]:
spark.sql("""
  CALL lake.system.rewrite_data_files(
    table => 'nyc.taxis',
    strategy => 'binpack',
    options => map(
      'min-input-files','1',                 -- be aggressive: rewrite even tiny groups
      'target-file-size-bytes','134217728',  -- ~128MB target size
      'partial-progress.enabled','true'      -- commit progress even if all tasks don't finish
    )
  )
""")

DataFrame[rewritten_data_files_count: int, added_data_files_count: int, rewritten_bytes_count: bigint, failed_data_files_count: int]

## Check Updated Partitions After Rewrite
Running this query again now shows actual partition values instead of {NULL}.
This confirms that the rewrite_data_files command applied the new day-based partitioning to existing data.

You should now see multiple partitions, each representing a different day from tpep_pickup_datetime.

In [18]:
spark.sql("SELECT partition, record_count, file_count, last_updated_snapshot_id  FROM nyc.taxis.partitions").show(truncate=False)


+------------+------------+----------+------------------------+
|partition   |record_count|file_count|last_updated_snapshot_id|
+------------+------------+----------+------------------------+
|{2025-04-30}|20          |1         |8962731662446708812     |
|{2025-05-01}|157140      |3         |8962731662446708812     |
|{2025-05-04}|142682      |3         |8962731662446708812     |
|{2025-05-05}|137188      |3         |8962731662446708812     |
|{2025-05-02}|147086      |3         |8962731662446708812     |
|{2025-05-03}|164004      |4         |8962731662446708812     |
|{2025-05-08}|165824      |4         |8962731662446708812     |
|{2025-05-09}|163691      |3         |8962731662446708812     |
|{2025-05-06}|144921      |3         |8962731662446708812     |
|{2025-05-07}|149451      |3         |8962731662446708812     |
|{2025-05-12}|128681      |3         |8962731662446708812     |
|{2025-05-13}|151498      |3         |8962731662446708812     |
|{2025-05-10}|159387      |4         |89

## Verify Rewrite in Table History
This query checks the nyc.taxis.history table to confirm that the data rewrite created a new snapshot.
You should see a new entry at the top, showing when the rewrite was committed and linking it to a new snapshot ID.

In [19]:
spark.sql("""-- verify it took effect
SELECT * FROM nyc.taxis.history ORDER BY made_current_at DESC;
""").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-08-11 16:01:41.168|8962731662446708812|4331219133962302771|true               |
|2025-08-11 16:01:21.92 |4331219133962302771|565187721077818959 |true               |
|2025-08-11 16:01:16.329|565187721077818959 |8062238580457074585|true               |
|2025-08-11 16:01:07.283|8062238580457074585|NULL               |true               |
|2025-08-11 16:00:45.126|4274936221537877971|8062238580457074585|false              |
|2025-08-11 16:00:43.386|8062238580457074585|NULL               |true               |
+-----------------------+-------------------+-------------------+-------------------+



## Roll Back to Original Snapshot (Before Partitioning)
This command reverts the nyc.taxis table back to the original snapshot (snapshot_id_1), before any partitioning or file rewrites were applied.

It’s part of testing partition evolution — showing that Iceberg lets you roll back to earlier table versions, even if the partitioning strategy has changed.

In [20]:
spark.sql(f""" 
CALL lake.system.rollback_to_snapshot('nyc.taxis', {snapshot_id_1})
""")

DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]

## Confirm Partition State After Rollback
After rolling back to snapshot_id_1, this query shows the partition state of the table has returned to {NULL}.
This means the table is now back to its unpartitioned state — as it was before any partitioning was added or data was rewritten.

It confirms that Iceberg’s rollback also restores the previous partition layout, not just the data.

In [21]:
spark.sql("SELECT partition, record_count, file_count, last_updated_snapshot_id  FROM nyc.taxis.partitions").show(truncate=False)


+---------+------------+----------+------------------------+
|partition|record_count|file_count|last_updated_snapshot_id|
+---------+------------+----------+------------------------+
|{NULL}   |8914805     |10        |8062238580457074585     |
+---------+------------+----------+------------------------+

